# Llama

In [1]:
from utils.model_setup import *
from utils.dataset_setup import *
from utils.experiment_setup import *
from utils.save_results import *
from utils.state_control import *
from utils.run_experiment import *

In [2]:
from importlib import reload

In [3]:
model_identifier = 'llama'
path_to = "./data/"
api_key = 1 # With value -1 it will change every 10 iterations
execution_beggining = 6

## Setup

In [4]:
"""
Setup State
"""
use_experiment_state = True

state = create_state(2, -1, -1, 1, 6, 0)
if use_experiment_state:
  state = retrieve_state(path_to, model_identifier)

{'Model': 2, 'Database': 1, 'Experimento': 6, 'Execucao': 2, 'MaxExecucao': 6, 'Batch': 21}


In [5]:
execute_experiment(state, model_identifier, path_to, api_key, execution_beggining)

Choosen Model: llama-3.2-90b-text-preview


 Setting model llama-3.2-90b-text-preview with API Key 1 


Dataset escolhido: essaysFullGrade
Experimento escolhido: exp6
Create exp6
Prompt template: 
Conheça as cinco competências cobradas pelo Inep na redação do ENEM:

1. Domínio da escrita formal da língua portuguesa

É avaliado se a redação do participante está adequada às regras de ortografia, como acentuação, ortografia, uso de hífen, emprego de letras maiúsculas e minúsculas e separação silábica. Ainda são analisadas a regência verbal e nominal, concordância verbal e nominal, pontuação, paralelismo, emprego de pronomes e crase.

São seis níveis de desempenho: 
200 pontos - Demonstra excelente domínio da modalidade escrita formal da língua portuguesa e de escolha de registro. Desvios gramaticais ou de convenções da escrita serão aceitos somente como excepcionalidade e quando não caracterizarem reincidência.
160 pontos - Demonstra bom domínio da modalidade escrita formal da língua port